In [18]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.autograd import Variable


In [3]:
import torch.cuda as cuda

device = torch.device("cuda:0" if cuda.is_available() else "cpu")

In [8]:
from torchvision import datasets, transforms

transformations = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = datasets.CIFAR10(root='./CIFAR10/', download=True, transform=transformations)
test_set = datasets.CIFAR10(root='./CIFAR10/', download=True, train=False, transform=transformations)

  0%|          | 0/170498071 [00:00<?, ?it/s]

 99%|█████████▉| 168927232/170498071 [00:09<00:00, 18480465.64it/s]

Extracting ./CIFAR10/cifar-10-python.tar.gz to ./CIFAR10/
Files already downloaded and verified


170500096it [00:20, 18480465.64it/s]                               

In [9]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_set, batch_size, shuffle=True, num_workers=2)

In [10]:
from torch.nn import functional as F
class ConvModel(nn.Module):
  # Your Code Here
    def __init__(self):
        super(ConvModel, self).__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size = 3, stride = 1, padding = 1)
        self.conv2 = nn.Conv2d(16,16,kernel_size = 3, stride = 1, padding = 1)
        self.maxp = nn.MaxPool2d(2)
        self.ln1 = nn.Linear(4096,64)
        self.ln2 = nn.Linear(64,10)
    
    def forward(self, x):
        input_size = x.size(0)
        x1 = F.relu(self.conv1(x))
        x2 = self.maxp(x1)
        x3 = F.relu(self.conv2(x2))
        x4 = x3.view(input_size, -1)
        x5 = F.relu(self.ln1(x4))
        x6 = self.ln2(x5)

        return F.log_softmax(x6)

In [12]:
model = ConvModel().to(device)

loss = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=0.001, weight_decay=0.01)



In [26]:
def train(model, train, test, loss_func, opt, num_epochs=15):
    
    
    for i in range(num_epochs):
        model.train()
        for batch_idx,(image,label) in enumerate(train):
            images, labels = Variable(image), Variable(label)
            opt.zero_grad()

            pred = model(images)
            loss = loss_func(pred, labels)
            loss.backward()
            opt.step()
            if batch_idx % 10 == 0:
                print ('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                num_epochs, batch_idx * len(image), len(train.dataset),
                100. * batch_idx / len(train), loss.item()))
            
            
        model.eval()
        for batch_idx,(image,label) in enumerate(test):
            test_loss = 0
            test_acc = 0
            image, label = image.to(device), label.to(device)
            output = model(image)
            test_loss += loss_func(output,label)
            prediction = torch.argmax(output,dim=1)
            test_acc += prediction.eq(label.data.view_as(prediction)).cpu().sum()
            
        test_loss/=len(test.dataset)
        print ('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testdataset),
        100. * test_acc / len(test.dataset)))

In [30]:
metrics = train(model, train_loader, test_loader, loss, optimizer, 5)